In [ ]:
# versions: pandas==2.1.1, numpy==1.26.4, scikit-learn==1.6.1, scikit-survival==0.24.1, lifelines==0.30.0, scikit-optimize==0.10.2, statsmodels==0.14.0

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.impute import SimpleImputer
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import RepeatedStratifiedKFold
from sksurv.metrics import concordance_index_censored,concordance_index_ipcw,brier_score,as_integrated_brier_score_scorer,cumulative_dynamic_auc,integrated_brier_score
from sksurv.ensemble import ExtraSurvivalTrees
from lifelines import CoxPHFitter

from sklearn.feature_selection import SequentialFeatureSelector
from skopt import BayesSearchCV
import skopt


from sksurv.nonparametric import kaplan_meier_estimator
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
FEATURE_SET = "rBL"#one of "BL", "BL+VOL", "BL+RAD", "BL+VOL+RAD", "rBL", "rBL+VOL", "rBL+RAD", "rBL+VOL+RAD"
DATASET_SEL = "CN+MCI" #one of "CN+MCI", "MCI"

In [ ]:
DATASET_TRAIN="./data/"+DATASET_SEL+"_"+FEATURE_SET+"_train.csv"
DATASET_TEST="./data/"+DATASET_SEL+"_"+FEATURE_SET+"_test.csv"

In [ ]:
MODEL_DIR = os.path.join("./results/ExtraSurvivalTreessw_"+DATASET_SEL+"_"+FEATURE_SET+"/")
filenameCSV=MODEL_DIR+"/resultsCV.csv"
trainingDSCSV=MODEL_DIR+"/training.csv"
testDSCSV=MODEL_DIR+"/test.csv"

In [ ]:
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [ ]:
train=pd.read_csv(DATASET_TRAIN)
test=pd.read_csv(DATASET_TEST)
train=train.set_index(["PTID","IMAGEUID"])
test=test.set_index(["PTID","IMAGEUID"])

In [ ]:
if not((FEATURE_SET=="BL")|(FEATURE_SET=="rBL")):
    non_MRI_features=["PTEDUCAT", "ADAS11", "ADAS13", "ADASQ4", "MMSE", "CDRSB", "CDGLOBAL", 
                      "LIMMTOTAL", "RAVLT_immediate", "RAVLT_learning", "RAVLT_forgetting", 
                      "RAVLT_perc_forgetting", "mPACCdigit", "mPACCtrailsB", "LDELTOT", 
                      "TRAB", "FAQ", "MOCA", "EcogPtMem", "EcogPtLang", "EcogPtVisspat", 
                      "EcogPtPlan", "EcogPtOrgan", "EcogPtDivatt", "EcogPtTotal", "EcogSPMem", 
                      "EcogSPLang", "EcogSPVisspat", "EcogSPPlan", "cogSPOrgan", "EcogSPDivatt",
                      "EcogSPTotal", "AGE", "PHS", "CIR", "Status", "Time", "PTETHCAT_Unknown", 
                      "PTETHCAT_Not Hisp/Latino", "PTETHCAT_Hisp/Latino", "PTRACCAT_Am Indian/Alaskan", 
                      "PTRACCAT_Asian", "PTRACCAT_Hawaiian/Other PI", "PTRACCAT_Black", "PTRACCAT_White", 
                      "PTRACCAT_More than one", "PTRACCAT_Unknown", "PTMARRY_Divorced", 
                      "PTMARRY_Married", "PTMARRY_Never married", "PTMARRY_Unknown", "PTMARRY_Widowed", 
                      "APOE4_0.0", "APOE4_1.0", "APOE4_2.0", "MagStrength_1.5", "MagStrength_3.0", 
                      "PTGENDER_Female", "PTGENDER_Male"]
    normalization_models=dict()
    for col in train.columns[~train.columns.isin(non_MRI_features)]:
        x=train["MagStrength_3.0"].astype(int).to_numpy().reshape((-1, 1))
        x_test=test["MagStrength_3.0"].astype(int).to_numpy().reshape((-1, 1))
        y = train[col].to_numpy()
        model = LinearRegression()
        model.fit(x, y)
        pred=model.predict(x)
        pred_test=model.predict(x_test)
        train.loc[:,col]=train[col]-pred
        test.loc[:,col]=test[col]-pred_test
        normalization_models[col]=model
    filename=MODEL_DIR+"normalization_models.sav"
    pickle.dump(normalization_models, open(filename, "wb"))

In [ ]:
selector = VarianceThreshold()
X=selector.fit_transform(train.drop(["Time","Status"],axis=1))

df_X=pd.DataFrame(X)

df_X.columns=selector.get_feature_names_out()
df_X=df_X.reset_index(drop=True)
df_X["PTID"]=train.index.get_level_values(0).tolist()
df_X["IMAGEUID"]=train.index.get_level_values(1).tolist()
df_X=df_X.set_index(["PTID","IMAGEUID"])
X_train=df_X

In [ ]:
test_filtered=test.filter(df_X.columns.tolist(),axis=1)
X_test=test_filtered

In [ ]:
imp = SimpleImputer(strategy="median").fit(X_train)
X_train_pre_imp=imp.transform(X_train)
X_test_pre_imp=imp.transform(X_test)

scaler = RobustScaler().fit(X_train_pre_imp)
X_train_pre=scaler.transform(X_train_pre_imp)
X_test_pre=scaler.transform(X_test_pre_imp)

In [ ]:
dict_preproc={}
dict_preproc["imputation"]=imp
dict_preproc["scaler"]=scaler
filename=MODEL_DIR+"preprocessing_entire_training.sav"
pickle.dump(dict_preproc, open(filename, "wb"))

In [ ]:
dataNamed=pd.DataFrame(X_train_pre,columns=X_train.columns)
dataNamed_test=pd.DataFrame(X_test_pre,columns=X_train.columns)

In [ ]:
dataNamed["PTID"]=train.index.get_level_values(0).tolist()
dataNamed["IMAGEUID"]=train.index.get_level_values(1).tolist()
dataNamed=dataNamed.set_index(["PTID","IMAGEUID"])

dataNamed_test["PTID"]=X_test.index.get_level_values(0).tolist()
dataNamed_test["IMAGEUID"]=X_test.index.get_level_values(1).tolist()
dataNamed_test=dataNamed_test.set_index(["PTID","IMAGEUID"])

In [ ]:
discr_ind=[]
i=0
for col in X_train.columns:
    if X_train[col].value_counts().shape[0]<=2:
        discr_ind.append(i)
    i+=1
for ind in discr_ind:
    col=imp.get_feature_names_out()[ind]
    dataNamed[col]=X_train_pre_imp[:,ind]
    dataNamed_test[col]=X_test_pre_imp[:,ind]
filename=MODEL_DIR+"discr_ind.sav"
pickle.dump(discr_ind, open(filename, "wb"))

In [ ]:
dataNamed_filtered=dataNamed
dataNamed_filtered_test=dataNamed_test
df=dataNamed_filtered
df_test=dataNamed_filtered_test

In [ ]:
discr=[]
for col in df.columns:
    if df[col].value_counts().shape[0]<=5:
        print(col)
        discr.append(True)
    else:
        discr.append(False)

In [ ]:
list_Y_train=list()
for i, dat in train.iterrows():
    statNew=False
    time=dat.Time
    if (dat.Status=="sCN") or (dat.Status=="sMCI") or (dat.Status=="CNtoMCI") or (dat.Status=="uMCINoAD") or (dat.Status=="uCNNoAD"):
        statNew=False
    else:
        statNew=True
    list_Y_train.append((statNew,time))

In [ ]:
list_Y_test=list()
for i, dat in test.iterrows():
    statNew=False
    time=dat.Time
    if (dat.Status=="sCN") or (dat.Status=="sMCI") or (dat.Status=="CNtoMCI") or (dat.Status=="uMCINoAD") or (dat.Status=="uCNNoAD"):
        statNew=False
    else:
        statNew=True
    list_Y_test.append((statNew,time))

In [ ]:
dt=np.dtype("bool,float")

y_train=np.array(list_Y_train,dtype=dt)

In [ ]:
status=[i[0] for i in y_train]
time_extracted=[i[1] for i in y_train]

In [ ]:
df_copy=df.copy()

df_copy.loc[:,"Time"]=time_extracted

df_copy.loc[:,"Event"]=status

df_copy.iloc[:,:-2].columns

res_list=[]

for col in df_copy.iloc[:,:-2].columns:
    cph = CoxPHFitter(penalizer=0.1)
    cph.fit(df_copy.filter([col,"Time","Event"],axis=1), duration_col="Time", event_col="Event")
    value=cph.concordance_index_
    res_list.append(value)
res=np.array(res_list)

In [ ]:
scores = [(df.columns[i],res[i]) for i in range(res.shape[0])]
scores.sort(key = lambda x : -x[1])

In [ ]:
scores.sort(key = lambda x : -x[1])

In [ ]:
def compute_vif(X):
     
    vif = pd.DataFrame()
    vif["Variable"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif[vif["Variable"]!="intercept"]
    return vif

In [ ]:
selected_features = []
for i in range(len(scores)):
    if len(selected_features) == 0:
        selected_features.append(scores[0][0])
        print(scores[0][0])
    else:
        feature_it = scores[i][0]
        df_new=df.filter(selected_features+[feature_it],axis=1)
        vifs=compute_vif(df_new)
        if not ((vifs.VIF>10).any() | ((df_new.corr()-np.identity(df_new.shape[1])).abs().max().max()>0.9)|(scores[i][1]<=0.0)):
            selected_features.append(feature_it)
            print(feature_it)
    if i%1000==0 :
        print(str(i)+": "+str(len(selected_features)))

In [ ]:
len(selected_features)

In [ ]:
selected_features

In [ ]:
df_X_sel_FS1=df.filter(selected_features,axis=1)

In [ ]:
df_X_sel_FS1.to_csv(MODEL_DIR+"features_PCA_filtered_train.csv")

In [ ]:
train_filtered_FS1=df_X_sel_FS1

In [ ]:
train_copy=train_filtered_FS1.copy()
train_copy=train_copy.reset_index(drop=True)


In [ ]:
index_pairs=[]
rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=20, random_state=412)
for i, (train_index, test_index) in enumerate(rkf.split(train,train.Status)):
    filtered_t=train.iloc[train_index]
    filtered_test=train.iloc[test_index]
    extracted_indices=train_copy[train_filtered_FS1.index.get_level_values(0).isin(filtered_t.index.get_level_values(0))].index
    test_part=train[train_filtered_FS1.index.get_level_values(0).isin(filtered_test.index.get_level_values(0))]
    test_part=test_part.sort_values("AGE")
    extracted_indices_test=train_copy[train_filtered_FS1.index.get_level_values(1).isin(test_part[~test_part.index.get_level_values(0).duplicated()].index.get_level_values(1))].index
    index_pairs.append((extracted_indices.tolist(),extracted_indices_test.tolist()))

In [ ]:
event_ind = [x[0] for x in y_train]
time_ind = [x[1] for x in y_train]

In [ ]:

times, survival_probs = kaplan_meier_estimator(~np.array(event_ind), np.array(time_ind))

weights = np.interp( np.array(time_ind), times, survival_probs)
weights = np.clip(weights, 0.01, 1.0) 
weights = 1 / weights

In [ ]:
clf = ExtraSurvivalTrees(random_state=474)

In [ ]:
def make_ipcw_brier_scorer(y, sample_weight, times, cv):
    def scorer(estimator, X_selected, y_selected):
        scores = []
        for train_idx, test_idx in cv:
            X_tr, X_te = X_selected[train_idx], X_selected[test_idx]
            y_tr, y_te = y[train_idx], y[test_idx]
            w_tr, w_te = sample_weight[train_idx], sample_weight[test_idx]

            estimator.fit(X_tr, y_tr, sample_weight=w_tr)
            surv_fns = estimator.predict_survival_function(X_te)
            preds = np.asarray([[fn(t) for t in times] for fn in surv_fns])

            score = score = integrated_brier_score(y_tr, y_te, preds, times)
            scores.append(score)
        return -np.mean(scores)  
    return scorer

scorer = make_ipcw_brier_scorer(y_train, weights, [4.0,8.0], index_pairs)


In [ ]:
Y_time=[i[1] for i in y_train]

In [ ]:
if train_filtered_FS1.shape[1]>10:
    sfs1 = SequentialFeatureSelector(
        estimator=clf,
        n_features_to_select=min(train_filtered_FS1.shape[1],11),
        scoring=scorer,
        cv=[(np.arange(len(y_train)), np.arange(len(y_train)))])

    sfs1 = sfs1.fit(train_filtered_FS1, y_train)


    selected_columns=train_filtered_FS1.columns[sfs1.support_].tolist()

    selected_columns

    train_filtered_FS2=train_filtered_FS1[selected_columns]
    test=df_test[selected_columns]
else:
    train_filtered_FS2=train_filtered_FS1
    test=df_test

In [ ]:
sfs2 = SequentialFeatureSelector(
    estimator=clf,
    n_features_to_select="auto",
    scoring=scorer,
    cv=[(np.arange(len(y_train)), np.arange(len(y_train)))], tol=0.0
)

sfs2 = sfs2.fit(train_filtered_FS2, y_train)

sfs2.feature_names_in_[sfs2.support_]


selected_columns=train_filtered_FS2.columns[sfs2.support_].tolist()

train_filtered_FS2=train_filtered_FS2[selected_columns]
test=df_test[selected_columns]

In [ ]:
train_filtered_FS2.to_csv(trainingDSCSV, header=True)
test.to_csv(testDSCSV, header=True)


In [ ]:
index_pairs=[]
rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=20, random_state=7312)
for i, (train_index, test_index) in enumerate(rkf.split(train,train.Status)):
    filtered_t=train.iloc[train_index]
    filtered_test=train.iloc[test_index]
    extracted_indices=train_copy[train_filtered_FS2.index.get_level_values(0).isin(filtered_t.index.get_level_values(0))].index
    test_part=train[train_filtered_FS2.index.get_level_values(0).isin(filtered_test.index.get_level_values(0))]
    test_part=test_part.sort_values("AGE")
    extracted_indices_test=train_copy[train_filtered_FS2.index.get_level_values(1).isin(test_part[~test_part.index.get_level_values(0).duplicated()].index.get_level_values(1))].index
    index_pairs.append((extracted_indices.tolist(),extracted_indices_test.tolist()))

In [ ]:
def ipcw_brier_scorer(estimator, X, y):
    scores = []
    for train_idx, test_idx in index_pairs:  
        X_tr, X_te = X.iloc[train_idx], X.iloc[test_idx]
        y_tr, y_te = y[train_idx], y[test_idx]
        w_tr, _ = weights[train_idx], weights[test_idx]

        estimator.fit(X_tr, y_tr, sample_weight=w_tr)
        surv_fns = estimator.predict_survival_function(X_te)
        preds = np.asarray([[fn(t) for t in [4.0, 8.0]] for fn in surv_fns])

        score = integrated_brier_score(y_tr, y_te, preds, [4.0, 8.0])
        scores.append(score)
    return -np.mean(scores)

In [ ]:
params = dict()
params["n_estimators"] = skopt.space.space.Integer(10,1000)
params["max_depth"] = skopt.space.space.Integer(5,20)
params["min_samples_split"] = skopt.space.space.Real(0.001,0.1)
params["min_samples_leaf"] = skopt.space.space.Real(0.001,0.1)
params["max_features"] = skopt.space.space.Categorical(["sqrt","log2"])
training=train_filtered_FS2


n_jobs=5

model = ExtraSurvivalTrees(random_state=474)

clf = BayesSearchCV(
    estimator=model,
    search_spaces=params,
    n_iter=100,
    cv=[(np.arange(len(y_train)), np.arange(len(y_train)))],
    n_jobs=-1,
    scoring=ipcw_brier_scorer,  
    optimizer_kwargs={
        "initial_point_generator": "lhs",
        "n_initial_points": 50,
        "acq_func": "LCB"
    },
    random_state=50,
    return_train_score=True
)
clf.fit(training, y_train)
filename=MODEL_DIR+"model_bayes_optimization.sav"
pickle.dump(clf, open(filename, "wb"))

In [ ]:
estim=clf.best_estimator_
scores = []
for train_idx, test_idx in index_pairs: 
    X_tr, X_te = training.iloc[train_idx], training.iloc[test_idx]
    y_tr, y_te = y_train[train_idx], y_train[test_idx]
    w_tr, _ = weights[train_idx], weights[test_idx]

    estim.fit(X_tr, y_tr, sample_weight=w_tr)
    surv_fns = estim.predict_survival_function(X_te)
    preds = np.asarray([[fn(t) for t in [4.0, 8.0]] for fn in surv_fns])

    score = integrated_brier_score(y_tr, y_te, preds, [4.0, 8.0])
    scores.append(score)

In [ ]:
df_res=pd.DataFrame(clf.cv_results_)
columns_new=df_res.columns[df_res.columns.str.startswith("param_")]
df_res_filtered=df_res.filter(columns_new.tolist()+["mean_test_score","std_test_score","mean_train_score","std_train_score"],axis=1)
df_res_filtered.sort_values("mean_test_score")

In [ ]:
survival_functions=clf.best_estimator_.predict_survival_function(training)
surv_4 = np.vstack([chf(4.0) for chf in survival_functions])
surv_8 = np.vstack([chf(8.0) for chf in survival_functions])


status=[i[0] for i in y_train]
time=[i[1] for i in y_train]

clf_chf_funcs = clf.best_estimator_.predict_cumulative_hazard_function(training, return_array=False)
clf_risk_scores_4 = np.vstack([chf(4.0) for chf in clf_chf_funcs])
clf_risk_scores_8 = np.vstack([chf(8.0) for chf in clf_chf_funcs])

cic_4=concordance_index_censored(status, time, clf_risk_scores_4[:,0].tolist())[0]
cic_8=concordance_index_censored(status, time, clf_risk_scores_8[:,0].tolist())[0]

cii_4=concordance_index_ipcw(y_train,y_train,clf_risk_scores_4[:,0].tolist(),tau=4.0)[0]
cii_8=concordance_index_ipcw(y_train,y_train,clf_risk_scores_8[:,0].tolist(),tau=8.0)[0]

auc_4=cumulative_dynamic_auc(y_train, y_train, clf_risk_scores_4[:,0].tolist(), 4.0)[1]
auc_8=cumulative_dynamic_auc(y_train, y_train, clf_risk_scores_8[:,0].tolist(), 8.0)[1]

bs_4=brier_score(y_train, y_train, surv_4[:,0], 4.0)[1][0]
bs_8=brier_score(y_train, y_train, surv_8[:,0], 8.0)[1][0]


print("cic_4: "+str(round(cic_4*100,3)))
print("cic_8: "+str(round(cic_8*100,3)))
print("cii_4: "+str(round(cii_4*100,3)))
print("cii_8: "+str(round(cii_8*100,3)))
print("auc_4: "+str(round(auc_4*100,3)))
print("auc_8: "+str(round(auc_8*100,3)))
print("bs_4: "+str(round(bs_4*100,3)))
print("bs_8: "+str(round(bs_8*100,3)))

In [ ]:
dt=np.dtype("bool,float")

y_test=np.array(list_Y_test,dtype=dt)

In [ ]:
survival_functions=clf.best_estimator_.predict_survival_function(test)
surv_4 = np.vstack([chf(4.0) for chf in survival_functions])
surv_8 = np.vstack([chf(8.0) for chf in survival_functions])


status=[i[0] for i in y_test]
time=[i[1] for i in y_test]

clf_chf_funcs = clf.best_estimator_.predict_cumulative_hazard_function(test, return_array=False)
clf_risk_scores_4 = np.vstack([chf(4.0) for chf in clf_chf_funcs])
clf_risk_scores_8 = np.vstack([chf(8.0) for chf in clf_chf_funcs])

cic_4=concordance_index_censored(status, time, clf_risk_scores_4[:,0].tolist())[0]
cic_8=concordance_index_censored(status, time, clf_risk_scores_8[:,0].tolist())[0]

cii_4=concordance_index_ipcw(y_train,y_test,clf_risk_scores_4[:,0].tolist(),tau=4.0)[0]
cii_8=concordance_index_ipcw(y_train,y_test,clf_risk_scores_8[:,0].tolist(),tau=8.0)[0]

auc_4=cumulative_dynamic_auc(y_train, y_test, clf_risk_scores_4[:,0].tolist(), 4.0)[1]
auc_8=cumulative_dynamic_auc(y_train, y_test, clf_risk_scores_8[:,0].tolist(), 8.0)[1]

bs_4=brier_score(y_train, y_test, surv_4[:,0], 4.0)[1][0]
bs_8=brier_score(y_train, y_test, surv_8[:,0], 8.0)[1][0]


print("cic_4: "+str(round(cic_4*100,3)))
print("cic_8: "+str(round(cic_8*100,3)))
print("cii_4: "+str(round(cii_4*100,3)))
print("cii_8: "+str(round(cii_8*100,3)))
print("auc_4: "+str(round(auc_4*100,3)))
print("auc_8: "+str(round(auc_8*100,3)))
print("bs_4: "+str(round(bs_4*100,3)))
print("bs_8: "+str(round(bs_8*100,3)))

In [ ]:
df=pd.DataFrame({"model":[MODEL_DIR.split("/")[2].split("_")[0]+"_sw"],"features":[FEATURE_SET],"problem":[DATASET_SEL],"concordance_index_ipcw_mean_CV":np.mean(scores),"concordance_index_ipcw_std_CV":np.std(scores),"concordance_index_censored_4":[cic_4],"concordance_index_censored_8":[cic_8],"concordance_index_ipcw_4":[cii_4],"concordance_index_ipcw_8":[cii_8],"cumulative_dynamic_auc_4": [auc_4],"cumulative_dynamic_auc_8": [auc_8],"brier_score_4":[bs_4],"brier_score_8":[bs_8]})
df.to_csv(MODEL_DIR+"res_table.csv")

In [ ]:
filename=MODEL_DIR+"y_test.sav"
pickle.dump(y_test, open(filename, "wb"))

In [ ]:
filename=MODEL_DIR+"y_train.sav"
pickle.dump(y_train, open(filename, "wb"))